In [3]:
import numpy as np
import matplotlib.pylab as plt

# 交差エントロピー

普通の交差エントロピーの実装

$\boldsymbol{y}$は，ワンホットエンコーディングされた，ベクトルである，\
よってyの総和は1になっている．

一つ目の例では．データ1個に対して損失を計算している．\
二つ目では，データ2個に対して損失を計算している．

各レコードの総和は1になっている．

In [4]:
def cross_entropy_error(y, t):
    print("元のデータサイズ", y.shape)
    print("元のデータサイズ", t.shape)
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    print("reshape後のデータサイズ", y.shape)
    print("reshape後のデータサイズ", t.shape)
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [6]:
cross_entropy_error(np.array([0.1, 0.2, 0.7]), np.array([0, 0, 1]))

元のデータサイズ (3,)
元のデータサイズ (3,)
reshape後のデータサイズ (1, 3)
reshape後のデータサイズ (1, 3)


np.float64(0.3566748010815999)

In [7]:
# 正解データが3次元配列の場合．
cross_entropy_error(np.array([[0.1, 0.2, 0.7], [0.8, 0.1, 0.1]]), np.array([[0, 0, 1], [1, 0, 0]]))

元のデータサイズ (2, 3)
元のデータサイズ (2, 3)
reshape後のデータサイズ (2, 3)
reshape後のデータサイズ (2, 3)


np.float64(0.2899091136979087)

ワンホットエンコーディングでないバージョン

In [8]:
def cross_entropy_error(y, t):
    print("元のデータサイズ", y.shape)
    print("元のデータサイズ", t.shape)
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    print("reshape後のデータサイズ", y.shape)
    print("reshape後のデータサイズ", t.shape)
    batch_size = y.shape[0]

    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [14]:
y = np.array([0.7, 0.1, 0.2])
t = np.array([2, 7, 0])
y[np.arange(y.shape[0]), t]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

なんかうまくいかないわ

In [9]:
cross_entropy_error(np.array([0.1, 0.2, 0.7]), np.array([0, 0, 1]))

元のデータサイズ (3,)
元のデータサイズ (3,)
reshape後のデータサイズ (1, 3)
reshape後のデータサイズ (1, 3)


np.float64(6.214605598423317)

# fとf()の違い．

以下のようなコードがある．

関数であればf()で渡すべきなのに，fだけで内部の変数が渡せているのは何故だろうか．

In [ ]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient


class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)

    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

x = np.array([0.6, 0.9])
t = np.array([0, 0, 1])

net = simpleNet()

f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W)

print(dW)

これは、Pythonにおける関数オブジェクトの重要な性質に関係しています。

### ステップバイステップの説明

1. まず、コードの該当部分を確認します：

In [ ]:
f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W)



2. この場合：
- `f` はラムダ関数で定義された関数オブジェクト
- `f` は変数名であり、関数そのものを参照
- `f()` は関数の実行を意味する

3. `numerical_gradient` 関数は以下のような動作をします：
- 第1引数として関数オブジェクト自体を受け取る
- 内部で必要に応じてその関数を実行する

### 重要なポイント

- `f` と `f()` の違い：
  - `f`: 関数オブジェクトそのもの（参照）
  - `f()`: 関数の実行

- Pythonでは関数は第一級オブジェクトとして扱われます：
  - 変数に代入可能
  - 他の関数の引数として渡すことが可能
  - 関数から返すことが可能

### 具体例



In [ ]:
# numerical_gradientの簡略化された内部実装例
def numerical_gradient(func, x):
    # funcは関数オブジェクト
    # 内部で必要に応じてfunc(x)として実行する
    result = func(x)  # ここで実際に関数を呼び出し
    return result

したがって、`numerical_gradient(f, net.W)`では、関数オブジェクト`f`自体を渡し、`numerical_gradient`関数の中で必要に応じて実行されるという仕組みになっています。

## 要点

fは関数のオブジェクト

fって変数に関数そのものが入っている．．実行するのは，他の関数の内部．

# りんごのやつの逆伝播

りんごの値段を$t$とする．んで，なんかりんごの値段は220円になったらしい．\
→この状況の時，$t$の微小な変化に対して，最終的な値段がどれだけ変化するかを求める．

という問題．

んで，中身では，以下のような処理が行われている．(順伝播)
$$
\begin{align*}
x_1 &= 2t\\
x_2 &= 1.1x_1\\
\text{total} &= x_2
\end{align*}
$$

これを逆伝播することによって答え($t$の微小な変化に対する，最終的な値段の変化)が求められる．\
実際にやってみる．
$$
\begin{align*}
\frac{d\text{total}}{dt} &= \frac{d\text{total}}{dx_2} \cdot \frac{dx_2}{dx_1} \cdot \frac{dx_1}{dt}\\
\\
\frac{d\text{total}}{dx_2} &= 1\\
\frac{dx_2}{dx_1} &= 1.1\\
\frac{dx_1}{dt} &= 2\\
\\
\frac{d\text{total}}{dt} &= 1 \cdot 1.1 \cdot 2 = 2.2
\end{align*}
$$
逆伝播によって求められた．

これのなにがすごいの？と思うかもしれない．

問題をもう一度考えてみよう．

りんごの値段を$t$とする(入力)．んで，なんかりんごの値段は220円になった(出力)\
今，入力と出力が与えられているね，これをニューラルネットの入力と出力と考える．

これだけの情報では，うまくパラメータを更新することができない．\
(本当は中身で複雑な処理が行われているけど，入力と出力のみからでは，いい感じのパラメータをみつけるための勾配が算出できない．)

しかし，途中の計算結果．$x_1，x_2$を残しておくことで，出力から逆向きに計算して，勾配を求めることができる．ということ．

本当は複雑なんだけど，途中の単純な計算を残しておくことで，複雑なニューラルネットのパラメータの更新に使うことができる．

# ReLU

In [37]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [38]:
ReLu = Relu()
out = ReLu.forward(np.array([[1, -2], [-3, 4]]))
print(out)
dx = ReLu.backward(out)
print(dx)

[[1 0]
 [0 4]]
[[1 0]
 [0 4]]


# np.dotの挙動の確認

入力：$X$，重み：$W$，バイアス：$B$，出力：$Y$とする．

$$
\begin{align*}
X &= \begin{bmatrix} X_1 \\ X_2 \end{bmatrix}\\
W &= \begin{bmatrix} W_{11} & W_{12} & W_{13} \\ W_{21} & W_{22} & W_{23} \end{bmatrix}\\
B &= \begin{bmatrix} B_1 \\ B_2 \\ B_3 \end{bmatrix}\\
Y &= \begin{bmatrix} Y_1 \\ Y_2 \\ Y_3 \end{bmatrix}
\end{align*}
$$

In [30]:
# randomのseedを固定
np.random.seed(0)
X = np.random.rand(2)
W = np.random.rand(2, 3)
B = np.random.rand(3)
print("input", X.shape)
print(X)
print("-" * 10)
print("weght", W.shape)
print(W)
print("-" * 10)
print("bias", B.shape)
print(B)

input (2,)
[0.5488135  0.71518937]
----------
weght (2, 3)
[[0.60276338 0.54488318 0.4236548 ]
 [0.64589411 0.43758721 0.891773  ]]
----------
bias (3,)
[0.96366276 0.38344152 0.79172504]


In [31]:
Y = np.dot(X, W) + B
print("output", Y.shape)
print(Y)

output (3,)
[1.75640404 0.99543849 1.66201908]


違和感がある．

$X$は2×1行列，$W$は2×3の行列と捉えられるのに，エラーが出ずに計算できるのはおかしい．

numpy.dot()の挙動を見てみる．

In [32]:
print("今回のやつ")
print(np.dot(X, W))
print("-" * 10)
print("計算として，正しくなるように転置した方")
print(np.dot(X.T, W))

今回のやつ
[0.79274128 0.61199697 0.87029404]
----------
計算として，正しくなるように転置した方
[0.79274128 0.61199697 0.87029404]


結果は同じになっている．

numpy.dot()はここをどう扱っているんだろう．

明示的にXの形状を，1×2行列にしてみる．
$$
X = \begin{bmatrix} X_1 & X_2 \end{bmatrix}
$$

In [38]:
np.random.seed(0)
X = np.random.rand(1, 2)
print("1×2 @ 2×3")
print(np.dot(X, W))
print("-" * 10)
print("2×1 @ 2×3")
print(np.dot(X.T, W))

1×2 @ 2×3
[[0.79274128 0.61199697 0.87029404]]
----------
2×1 @ 2×3


ValueError: shapes (2,1) and (2,3) not aligned: 1 (dim 1) != 2 (dim 0)

明示的にXの形状を，2×1行列にしてみる．
$$
X = \begin{bmatrix} X_1 \\ X_2 \end{bmatrix}
$$

In [39]:
np.random.seed(0)
X = np.random.rand(2, 1)
print("1×2 @ 2×3")
print(np.dot(X.T, W))
print("-" * 10)
print("2×1 @ 2×3")
print(np.dot(X, W))

1×2 @ 2×3
[[0.79274128 0.61199697 0.87029404]]
----------
2×1 @ 2×3


ValueError: shapes (2,1) and (2,3) not aligned: 1 (dim 1) != 2 (dim 0)

明示的に形状を指定するとエラーが出た．

np.random.rand(n)で生成する一次元配列は，n個の要素を持つ配列として扱われる．
だから都合の良い形状としてとられるぽいので，あんま気にしなくて良さそう．

# 内積の逆伝播

参考：[Affineレイヤの逆伝播を地道に成分計算する](https://qiita.com/yuyasat/items/d9cdd4401221df5375b6)

入力：$X$，重み：$W$，バイアス：$B$，出力：$Y$とする．

$$
\begin{align*}
X &= \begin{bmatrix} X_1 & X_2 \end{bmatrix}\\
W &= \begin{bmatrix} W_{11} & W_{12} & W_{13} \\ W_{21} & W_{22} & W_{23} \end{bmatrix}\\
B &= \begin{bmatrix} B_1 & B_2 & B_3 \end{bmatrix}\\
Y &= \begin{bmatrix} Y_1 & Y_2 & Y_3 \end{bmatrix}
\end{align*}
$$

Xはただの一次元配列だからこうする意味もあまりないけど．違和感があるので明示的に行列の形状を揃えておく．

損失関数を$L$とおく．

今回内積の逆伝播がどうなるかを見てみたい．

逆伝播の流れは以下の通り．
- 損失関数$L$を$Y$で微分
- $B$は，$Y$に対して加算されるだけなので，$B$に関しては，$L$を$Y$で微分した値がそのまま逆伝播する．
ここまでを数式で表すとこんな感じ．
$$
\begin{align*}
\frac{\partial L}{\partial Y} &= \begin{bmatrix} \frac{\partial L}{\partial Y_1} & \frac{\partial L}{\partial Y_2} & \frac{\partial L}{\partial Y_3} \end{bmatrix}
\end{align*}
$$
これだけ．これは次の材料になる．

ここから$X$の逆伝播，$W$の逆伝播を求める．

- $X$の逆伝播
損失関数に対しての$X$の偏微分．$L$を$X$の各要素について微分した形になっている．\
$X$の形状は，1×2行列なので，その形状を維持する．
$$
\begin{align*}
\frac{\partial L}{\partial X} &= \begin{bmatrix}\frac{\partial L}{\partial x_1} & \frac{\partial L}{\partial x_2}\end{bmatrix}
\end{align*}
$$
$\frac{\partial L}{\partial x_1}$は直接計算できない．なぜなら間に$Y$を挟んでいるから．\
連鎖律を使って計算する．
$$
\begin{align*}
\frac{\partial L}{\partial Y}\frac{\partial Y}{\partial X} &=
\begin{bmatrix}
\frac{\partial L}{\partial Y}\frac{\partial Y}{\partial x_1} & \frac{\partial L}{\partial Y}\frac{\partial Y}{\partial x_2}
\end{bmatrix}
\end{align*}
$$
$Y$が$X$を使って直接計算してるので，$\frac{\partial Y}{\partial x_1}$は計算できる．\
しかも，$\frac{\partial L}{\partial Y}$はすでに計算済みなので，これを使える．
$$
Y=XWより，\\
\begin{align*}
\frac{\partial Y}{\partial x_1} &=
\begin{bmatrix} \frac{\partial{Y_1}}{\partial{x_1}} & \frac{\partial{Y_2}}{\partial{x_1}} & \frac{\partial{Y_3}}{\partial{x_1}} \end{bmatrix}
\end{align*}
$$
ちなみに，$x_1$はスカラーなので，偏微分した後のベクトルは列でも行でもどっちでもいい．

### 偏微分の話
$a$(スカラー)を$\boldsymbol{b}$(ベクトル)で微分した結果は，$\boldsymbol{b}$の形状に依存する．
$$
\boldsymbol{a}\in \mathbb{R}\\
\boldsymbol{b}\in \mathbb{R}^m

\\
\frac{\partial{a}}{\partial{\boldsymbol{b}}} =
\begin{bmatrix}
\frac{\partial{a}}{\partial{b_1}} & \frac{\partial{a}}{\partial{b_2}} & \cdots & \frac{\partial{a}}{\partial{b_m}}
\end{bmatrix}
$$

$\boldsymbol{a}$(ベクトル)を$b$(スカラー)で微分した結果は，aの形状に依存する．
$$
\boldsymbol{a}\in \mathbb{R}^n\\
\boldsymbol{b}\in \mathbb{R}

\\
\frac{\partial{\boldsymbol{a}}}{\partial{b}} =
\begin{bmatrix}
\frac{\partial{a_1}}{\partial{b}} & \frac{\partial{a_2}}{\partial{b}} & \cdots & \frac{\partial{a_n}}{\partial{b}}
\end{bmatrix}
$$

$\boldsymbol{a}$(ベクトル)を$\boldsymbol{b}$(ベクトル)で微分した結果は，ヤコビ行列になる．
$$

\boldsymbol{a}\in \mathbb{R}^n\\
\boldsymbol{b}\in \mathbb{R}^m

\\

\mathbf{J} = \frac{\partial \mathbf{a}}{\partial \mathbf{b}} =
\begin{bmatrix}
\frac{\partial a_1}{\partial b_1} & \frac{\partial a_1}{\partial b_2} & \cdots & \frac{\partial a_1}{\partial b_m} \\
\frac{\partial a_2}{\partial b_1} & \frac{\partial a_2}{\partial b_2} & \cdots & \frac{\partial a_2}{\partial b_m} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{\partial a_n}{\partial b_1} & \frac{\partial a_n}{\partial b_2} & \cdots & \frac{\partial a_n}{\partial b_m}
\end{bmatrix}
$$